# Data Preprocessing

In [1]:
import csv
import pandas as pd
with open('training.csv','r') as train_data:
    reader1 = csv.DictReader(train_data)
    datas = [row for row in reader1]
## impord csv and pandas for data opration
## read the training file

In [2]:
import random

#random.shuffle(datas)

## random the data for testing

In [3]:
n = len(datas)//3

test_set = datas[0:n]
train_set = datas[n:]
## divied the data by 3:6 by using //

In [4]:
## the first method to change string to integer
## but I did not use it cause the huge number will influence the accuracy
def atoi(s):
    s = s[::-1]
    num = 0
    for i, v in enumerate(s):
        offset = ord(v) - ord('0')
        num += offset * (10 ** i)
    return float(num)

In [5]:
def Mapping(In_data):
    buying_mapping = {'low':0,'med':1,'high':2,'vhigh':3}
    maint_mapping = {'low':0,'med':1,'high':2,'vhigh':3}
    doors_mapping = {'2':2, '3':3, '4':4, '5more':5}
    persons_mapping = {'2':2, '4':4, 'more':5}
    lug_boot_mapping = {'small':0, 'med':1, 'big':2}
    safety_mapping = {'low':0, 'med':1, 'high':2}
    In_data['buying'] = In_data['buying'].map(buying_mapping)
    In_data['maint'] = In_data['maint'].map(maint_mapping)
    In_data['doors'] = In_data['doors'].map(doors_mapping)
    In_data['persons'] = In_data['persons'].map(persons_mapping)
    In_data['lug_boot'] = In_data['lug_boot'].map(lug_boot_mapping)
    In_data['safety'] = In_data['safety'].map(safety_mapping)

## function to change all string value to integer

In [6]:
train_set = pd.DataFrame(train_set,columns = ["buying","maint","doors","persons","lug_boot","safety","evaluation"])
Mapping(train_set)
## change all string value to integer in train_set
test_set = pd.DataFrame(test_set,columns = ["buying","maint","doors","persons","lug_boot","safety","evaluation"])
Mapping(test_set)
## change all string value to integer in test_set

In [7]:
train_set = train_set.to_dict(orient='records')
## change the data from DataFrame to dictionary
test_set = test_set.to_dict(orient='records')
## change the data from DataFrame to dictionary

In [8]:
## a function using Euclidean algorithm to calculate the distance
def Distance(v1,v2):
    res = 0
    ## the original distance is 0
    for key in ("buying", "maint", "doors", "persons", "lug_boot", "safety"):
        ## using the value except 'evaluation'
        res+=(float((v1[key])) - float((v2[key])))**2

    return res**0.5

# KNN

In [9]:
K = 5
## by testing from k = (0 to 20), 5 is the best K in this model

## the KNN mean function
def KNN(data):
    res = [
        {"result":train['evaluation'],"distance":Distance(data,train)}
        for train in train_set
    ]
    ## add the feature label
    
    res_sorted = sorted(res,key=lambda item:item['distance'])
    ## sort in ascending order

    res_K = res_sorted[0:K]
    ## Using K to divide the data set
    
    result = {'acc':0,'unacc':0,'good':0}
    ## set the original weight
    
    sum = 0
    for r in res_K:
        sum+=r['distance']
        ## scan ever distance in res_K
        
    for r in res_K:
        result[r['result']]+=1-r['distance']/sum
        ## add the weight in ever scan
        
    if result['acc']>result['unacc'] and result['acc']>result['good']:
        return 'acc'
    elif result['acc']>result['unacc']and result['acc']<result['good']:
        return 'good'
    else:
        return 'unacc'
    ## produce the result

In [10]:
import time
## add time to calculate the running time
time_start = time.time()
## markdown the start time

correct = 0
## set a value to count the number of correct
for test in test_set:
    result1 = test['evaluation']
    result2 = KNN(test)
    ## get the result of prediction and the result of test_set
    
    if result1 == result2:
        correct+=1
    ## if the result is same which means it is correct

time_end = time.time()
## markdown the end time

print('Accuracy:{:.2f}%'.format(100*correct/len(test_set)))
## print the accuracy
print('Time:{:.2f}s'.format(time_end - time_start))
## print the running time

Accuracy:94.13%
Time:0.71s


In [11]:
with open('test.csv','r') as test_data:
    reader1 = csv.DictReader(test_data)
    test_datas = [row for row in reader1]
## read test.csv
df_test_datas = pd.DataFrame(test_datas,columns = ["buying","maint","doors","persons","lug_boot","safety"])
## change the test data set to DataFrame

result = df_test_datas

Mapping(df_test_datas)
## mapping the test data
test_datas_dict = df_test_datas.to_dict(orient='records')
## change DataFrame to dictionary

for test in test_datas_dict:
    pre_evaluation = KNN(test)
    ## run KNN model with test set



In [12]:
result = pd.DataFrame(test_datas,columns = ["buying","maint","doors","persons","lug_boot","safety"])
result['evaluation'] = pre_evaluation
## fill the result in the original test data set
result

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,low,med,4,2,big,low,unacc
1,vhigh,med,2,2,med,high,unacc
2,vhigh,high,4,4,big,high,unacc
3,high,vhigh,4,2,small,med,unacc
4,vhigh,high,5more,2,small,high,unacc
...,...,...,...,...,...,...,...
328,high,low,2,more,big,low,unacc
329,high,med,4,2,big,low,unacc
330,high,med,4,4,big,high,unacc
331,high,med,2,4,med,low,unacc
